<a href="https://colab.research.google.com/github/shivang2607/Machine-Learning/blob/main/My_first_GPT_model_Recipy_(generation_with_attention).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import pandas as pd
import numpy as np
import json



In [2]:
# data = pd.read_json('/content/drive/MyDrive/Kaggle/full_format_recipes.json')
# data.head()


,directions,fat,date,categories,calories,desc,protein,rating,title,ingredients,sodium
0,"[1. Place the stock, lentils, celery, carrot, ...",7.0,2006-09-01 04:00:00+00:00,"[Sandwich, Bean, Fruit, Tomato, turkey, Vegeta...",426.0,None,30.0,2.500,"Lentil, Apple, and Turkey Wrap","[4 cups low-sodium vegetable or chicken stock,...",559.0
1,[Combine first 9 ingredients in heavy medium s...,23.0,2004-08-20 04:00:00+00:00,"[Food Processor, Onion, Pork, Bake, Bastille D...",403.0,This uses the same ingredients found in boudin...,18.0,4.375,Boudin Blanc Terrine with Red Onion Confit,"[1 1/2 cups whipping cream, 2 medium onions, c...",1439.0
2,[In a large heavy saucepan cook diced fennel a...,7.0,2004-08-20 04:00:00+00:00,"[Soup/Stew, Dairy, Potato, Vegetable, Fennel, ...",165.0,None,6.0,3.750,Potato and Fennel Soup Hodge,"[1 fennel bulb (sometimes called anise), stalk...",165.0
3,[Heat oil in heavy large skillet over medium-h...,NaN,2009-03-27 04:00:00+00:00,"[Fish, Olive, Tomato, Sauté, Low Fat, Low Cal,...",NaN,The Sicilian-style tomato sauce has tons of Me...,NaN,5.000,Mahi-Mahi in Tomato Olive Sauce,"[2 tablespoons extra-virgin olive oil, 1 cup c...",NaN
4,[Preheat oven to 350°F. Lightly grease 8x8x2-i...,32.0,2004-08-20 04:00:00+00:00,"[Cheese, Dairy, Pasta, Vegetable, Side, Bake, ...",547.0,None,20.0,3.125,Spinach Noodle Casserole,"[1 12-ounce package frozen spinach soufflé, th...",452.0


In [39]:
from IPython.display import HTML, display

def my_css():
   display(HTML("""<style>table.dataframe td{white-space: nowrap;}</style>"""))

get_ipython().events.register('pre_run_cell', my_css)

In [2]:
with open('/content/drive/MyDrive/Kaggle/full_format_recipes.json') as json_data:
    recipe_data = json.load(json_data)
filtered_data = ['Recipe for ' + x['title']+ ' | ' + ' '.join(x['directions'])
for x in recipe_data
if 'title' in x
and x['title'] is not None
and 'directions' in x
and x['directions'] is not None
]


In [3]:
# We would also like to tokenize punctuations as they tell us wether the sentence is exclamation or question etc
import string
import re
def pad_punctuation(s):
    s = re.sub(f"([{string.punctuation}])", r' \1 ', str(s))
    s = re.sub(' +', ' ', str(s))
    return s

text_data = [pad_punctuation(x) for  x in filtered_data]

In [ ]:
#convert text_data to tensorflow dataset
text_data=pd.DataFrame(text_data,columns=["data"])
text_ds = tf.data.Dataset.from_tensor_slices(text_data["data"]).batch(32).shuffle(1000)
text_vec = tf.keras.layers.TextVectorization(
    standardize="lower",
    max_tokens=10000,
    output_sequence_length = 250+1,
    output_mode='int',
)
text_vec.adapt(text_ds)
for ele in text_ds:
    print(ele)
    break

In [5]:
vocab=text_vec.get_vocabulary()

def prepare_inputs(text):
    text = tf.expand_dims(text, -1)
    tokenized_sentences = text_vec(text)
    x = tokenized_sentences[:, :-1]
    y = tokenized_sentences[:, 1:]
    return x, y
train_ds = text_ds.map(prepare_inputs)


In [6]:
def causal_attention_mask(batch_size, n_dest, n_src, dtype):
    i = tf.range(n_dest)[:, None]
    j = tf.range(n_src)
    m = i >= j - n_src + n_dest
    mask = tf.cast(m, dtype)
    mask = tf.reshape(mask, [1, n_dest, n_src])
    mult = tf.concat(
    [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)], 0
    )
    return tf.tile(mask, mult)
np.transpose(causal_attention_mask(1, 10, 10, dtype = tf.int32)[0])

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]], dtype=int32)

In [7]:

class TransformerBlock(layers.Layer):
    def __init__(self, num_heads, key_dim, embed_dim, ff_dim, dropout_rate=0.1):
        super(TransformerBlock, self).__init__()
        self.num_heads = num_heads
        self.key_dim = key_dim
        self.embed_dim = embed_dim
        self.ff_dim = ff_dim
        self.dropout_rate = dropout_rate
        self.attn = layers.MultiHeadAttention(num_heads, key_dim, output_shape = embed_dim, use_bias=False)
        self.dropout_1 = layers.Dropout(self.dropout_rate)
        self.ln_1 = layers.LayerNormalization(epsilon=1e-6)
        self.ffn_1 = layers.Dense(self.ff_dim, activation="relu")
        self.ffn_2 = layers.Dense(self.embed_dim)
        self.dropout_2 = layers.Dropout(self.dropout_rate)
        self.ln_2 = layers.LayerNormalization(epsilon=1e-6)
    def call(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size = input_shape[0]
        seq_len = input_shape[1]
        causal_mask = causal_attention_mask(batch_size, seq_len, seq_len,
        tf.bool)
        attention_output, attention_scores = self.attn(inputs, inputs,attention_mask=causal_mask, return_attention_scores=True)
        attention_output = self.dropout_1(attention_output)
        out1 = self.ln_1(inputs + attention_output)
        ffn_1 = self.ffn_1(out1)
        ffn_2 = self.ffn_2(ffn_1)
        ffn_output = self.dropout_2(ffn_2)
        return (self.ln_2(out1 + ffn_output), attention_scores)


In [8]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.maxlen = maxlen
        self.vocab_size =vocab_size
        self.embed_dim = embed_dim
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [11]:
# MY FIRST GPT MODEL
MAX_LEN = 200
VOCAB_SIZE = 10000
EMBEDDING_DIM = 256
N_HEADS = 2
KEY_DIM = 256
FEED_FORWARD_DIM = 256

inputs = layers.Input(shape=(None,), dtype=tf.int32)
x = TokenAndPositionEmbedding(MAX_LEN, VOCAB_SIZE, EMBEDDING_DIM)(inputs)
x, attention_scores = TransformerBlock(N_HEADS, KEY_DIM, EMBEDDING_DIM,FEED_FORWARD_DIM)(x)
x, attention_scores = TransformerBlock(N_HEADS, KEY_DIM, EMBEDDING_DIM,FEED_FORWARD_DIM)(x)
x = layers.Dense(128, activation="relu", kernel_initializer="he_normal")(x)
outputs = layers.Dense(VOCAB_SIZE, activation = 'softmax')(x)
gpt = models.Model(inputs=inputs, outputs=[outputs, attention_scores])
gpt.compile("adam", loss=[tf.keras.losses.SparseCategoricalCrossentropy(), None])


In [12]:
# print(tf.shape(list(text_ds.as_numpy_iterator())))
history = gpt.fit(train_ds, epochs=5, callbacks=[tf.keras.callbacks.ModelCheckpoint(
    "/content/drive/MyDrive/Model_chekpoints_logs/weights_of_first_gpt_model(Recipy_generation)",
    save_weights_only=True,
    save_best_only=True,
    mode="max",
    monitor="accuracy")])

Epoch 1/5
629/629 [==============================] - ETA: 0s - loss: 2.3536 - dense_11_loss: 2.3536

629/629 [==============================] - 86s 125ms/step - loss: 2.3536 - dense_11_loss: 2.3536
Epoch 2/5
629/629 [==============================] - ETA: 0s - loss: 1.6567 - dense_11_loss: 1.6567

629/629 [==============================] - 55s 88ms/step - loss: 1.6567 - dense_11_loss: 1.6567
Epoch 3/5
629/629 [==============================] - ETA: 0s - loss: 1.5050 - dense_11_loss: 1.5050

629/629 [==============================] - 53s 84ms/step - loss: 1.5050 - dense_11_loss: 1.5050
Epoch 4/5
629/629 [==============================] - ETA: 0s - loss: 1.4215 - dense_11_loss: 1.4215

629/629 [==============================] - 53s 84ms/step - loss: 1.4215 - dense_11_loss: 1.4215
Epoch 5/5
629/629 [==============================] - ETA: 0s - loss: 1.3629 - dense_11_loss: 1.3629

629/629 [==============================] - 53s 85ms/step - loss: 1.3629 - dense_11_loss: 1.3629


In [60]:
class TextGenerator(tf.keras.callbacks.Callback):
    def __init__(self, model, index_to_word, top_k=10):
        self.index_to_word = index_to_word
        self.model=model
        self.word_to_index = {word: index for index, word in
        enumerate(index_to_word)}

    def sample_from(self, probs, temperature):
        probs = probs ** (1 / (temperature+0.0001))
        probs = probs / np.sum(probs)
        return np.random.choice(len(probs), p=probs), probs
    def generate(self, start_prompt, max_tokens, temperature):
        start_tokens = [self.word_to_index.get(x, 1) for x in start_prompt.split()]
        sample_token = None
        info = []
        while len(start_tokens) < max_tokens and sample_token != 0:
            x = np.array([start_tokens])
            y = self.model.predict(x)
            sample_token, probs = self.sample_from(y[0][0][-1], temperature)
            info.append({'prompt': start_prompt , 'word_probs': probs})
            start_tokens.append(sample_token)
            start_prompt = start_prompt + ' ' + self.index_to_word[sample_token]
            print(f"\ngenerated text:\n{start_prompt}\n")
        return info
    def on_epoch_end(self, epoch, logs=None):
        self.generate("recipe for", max_tokens = 100, temperature = 1.0)

In [61]:
text_generator = TextGenerator(gpt, text_vec.get_vocabulary())

In [62]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [64]:

info = text_generator.generate("recipe for ", max_tokens=150, temperature=0.5)

1/1 [==============================] - 0s 44ms/step

generated text:
recipe for  grilled

1/1 [==============================] - 0s 55ms/step

generated text:
recipe for  grilled halibut

1/1 [==============================] - 0s 56ms/step

generated text:
recipe for  grilled halibut with

1/1 [==============================] - 0s 39ms/step

generated text:
recipe for  grilled halibut with tomato

1/1 [==============================] - 0s 44ms/step

generated text:
recipe for  grilled halibut with tomato and

1/1 [==============================] - 0s 23ms/step

generated text:
recipe for  grilled halibut with tomato and onion

1/1 [==============================] - 0s 23ms/step

generated text:
recipe for  grilled halibut with tomato and onion |

1/1 [==============================] - 0s 21ms/step

generated text:
recipe for  grilled halibut with tomato and onion | prepare

1/1 [==============================] - 0s 27ms/step

generated text:
recipe for  grilled halibut with tomato and 

In [94]:
output, attention_scores = gpt.predict(text_vec(["recipe for "]))
attention_scores[0][0][9]           # first dimension is 1, second dimension is 2 i.e. number of heads,
                                    # and 3rd and 4th dimension is 251-251 kyunki max_len = 251 rkhi h to
                                    # to mtlb konsa index ka word konse index pr sbse jyada focus kr rha h
                                    # kyunki initially 1 word se start krte h to candidate word first word
                                    # pr hi attention de paata h and then fir generate hone k baad 1 word add
                                    # ho gaya prompt m to teesra word pehle 2 pr uska attention distributd hota
                                    # h, aise hi 13th generated word pichle 12 words pr attention de paayega


1/1 [==============================] - 0s 40ms/step


array([0.0410438 , 0.14003523, 0.0406408 , 0.07763904, 0.07841241,
       0.08503987, 0.09607931, 0.09328586, 0.17420892, 0.17361471,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     